In [ ]:
from utils import *

In [ ]:
conf_dict_with = {}
desc_dict_with = {}

conf_dict_without = {}
desc_dict_without = {}

total_with, total_without = 0, 0

for i in list(range(58)):
    print(f"----- Salt #{i} -----")

    with_ce, without_ce, best_ce = make_best_ensemble(i)
    n_atoms = with_ce.elements.shape[0]

    if with_ce.n_conformers > 100 or without_ce.n_conformers > 100:
        rmsd_thresh = 1.0
        energy_thresh = 1.5
    else:
        rmsd_thresh = 0.5
        energy_thresh = 3.0
    
    with_ce = trim_conformers(with_ce, rmsd_thresh=rmsd_thresh, energy_thresh=energy_thresh)
    conf_dict_with[i] = (with_ce, with_ce.n_conformers)
    total_with += with_ce.n_conformers

    without_ce = trim_conformers(without_ce, rmsd_thresh=rmsd_thresh, energy_thresh=energy_thresh)
    conf_dict_without[i] = (without_ce, without_ce.n_conformers)
    total_without += without_ce.n_conformers

    print(f"   Atoms: {n_atoms}, rmsd: {rmsd_thresh}, energy: {energy_thresh}, with: {with_ce.n_conformers}, without: {without_ce.n_conformers}")

    dump_conformers(with_ce, i, withh=True)
    dump_conformers(without_ce, i, withh=False)

    gen_hess(i, 64, withh=True)
    gen_hess(i, 64, withh=False)

    with_bw = with_ce.boltzmann_weights().T
    without_bw = without_ce.boltzmann_weights().T

    with_ensemble_desc = get_ensemble_descriptors(i, withh=True, dispersion=True, SASA=True, XTB=False, LFCs=None)
    without_ensemble_desc = get_ensemble_descriptors(i, withh=False, dispersion=True, SASA=True, XTB=False, LFCs=None)
    n_conformers, n_descriptors = len(list(with_ensemble_desc)), len(list(with_ensemble_desc[0].values()))

    bw_desc_with = boltz_weight_desc(with_ensemble_desc, with_bw)
    bw_desc_without = boltz_weight_desc(without_ensemble_desc, without_bw)
    
    desc_dict_with[i] = bw_desc_with
    desc_dict_without[i] = bw_desc_without

with_df = pd.DataFrame.from_dict(desc_dict_with, orient="index").round(2)
without_df = pd.DataFrame.from_dict(desc_dict_without, orient="index").round(2)

with_df.to_csv("descriptors_with_chlorine.csv", index=False)
without_df.to_csv("descriptors_without_chlorine.csv", index=False)

In [ ]:
with_df